In [2]:
#Title: Christian's Color Collection Catalog
#Purpose: This program prompts the user to select a color and then offers several swatches of similar colors so they can choose their favorite. Then, the user has the option to save their selection to a database.
#Warning: Don't select Black. Black is a shade, not a color. So is white...

import tkinter as tk
from tkinter import ttk
import sqlite3
import os

#Create SQL database
def create_database():
    current_dir = os.getcwd()
    db_path = os.path.join(current_dir, "color_selections.db")

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS selections (
            id INTEGER PRIMARY KEY,
            color TEXT NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

#Save the selection to the database
def save_to_database(selected_color):
    current_dir = os.getcwd()
    db_path = os.path.join(current_dir, "color_selections.db")

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO selections (color) VALUES (?)", (selected_color,))
    conn.commit()
    conn.close()
    print(f"Saved {selected_color} to database.")

#Main window
main = tk.Tk()
main.title("Favorite Color")

#Initialize the database
create_database()

#Variables
color = tk.StringVar()

#Color selection combobox
color_pick = ttk.Combobox(main, width = 40, textvariable = color)
color_pick['values'] = ('Aquamarine', 'Black', 'Blue', 'Brown', 'Green', 'Grey', 'Orange', 'Pink', 'Purple', 'Red', 'Teal', 'White', 'Yellow')

#Labels and entry
label_color = tk.Label(main, text = "Which of these is closest \nto your favorite color?", font = ('calibre', 14, 'bold'))

#Function to display swatches
def display_swatches(popup, user_color):
    for widget in popup.winfo_children():
        widget.destroy()

    #Custom messages and behavior for specific colors
    if user_color.lower() == "black":
        msg = tk.Label(popup, text = "Really? Your favorite color is black? Who are you, Batman? Pick something else, Dark Knight.", font = ('calibre', 14, 'italic'))
        msg.pack(pady = 10)

        def close_popups():
            popup.destroy()

        close_button = tk.Button(popup, text = "Fine...", command = close_popups)
        close_button.pack(pady = 10)
        return

    elif user_color.lower() == "grey":
        msg = tk.Label(popup, text = "Oh, brother. A little depressing, don't ya think? Fine, go ahead.", font = ('calibre', 14, 'italic'))
        msg.pack(pady = 10)

    elif user_color.lower() == "white":
        msg = tk.Label(popup, text = "White, huh? Kinda boring, but I'll allow it.", font = ('calibre', 14, 'italic'), bg = 'black', fg = 'white')
        msg.pack(pady = 10)

    elif user_color.lower() == "yellow":
        msg = tk.Label(popup, text = "You selected: Yellow", font = ('calibre', 14, 'normal'), bg = 'black', fg = 'yellow')
        msg.pack(pady=10)

    else:
        msg = tk.Label(popup, text = f"You selected: {user_color}", font = ('calibre', 14, 'normal'), fg = user_color)
        msg.pack(pady = 10)

    #Display a message for swatches
    message_label = tk.Label(popup, text = "Now select a swatch:", font = ('calibre', 14, 'normal'))
    message_label.pack(pady = 10)

    #Swatches
    color_swatches = {
        'Aquamarine': ['Aquamarine', 'Medium Aquamarine', 'Turquoise', 'Dark turquoise', 'Cyan'],
        'Black': ['Black', 'Black', 'Black', 'Black', 'Black'],
        'Blue': ['Blue', 'Sky Blue', 'Dodger Blue', 'Steel Blue', 'Deep Sky Blue'],
        'Brown': ['Brown', 'Saddle Brown', 'Sienna4', 'Peru', 'Chocolate'],
        'Green': ['Green', 'Forest Green', 'Sea Green', 'Lime', 'Spring Green'],
        'Grey': ['Grey', 'Dim Gray', 'Slate Gray', 'Light Slate Gray', 'Light Gray'],
        'Orange': ['Orange', 'Dark Orange', 'Coral', 'Tomato', 'Orange Red'],
        'Pink': ['Pink', 'Hot Pink', 'Deep Pink', 'Deep Pink3', 'Medium Violet Red'],
        'Purple': ['Purple', 'Dark Orchid', 'Medium Purple', 'Purple4', 'Indigo'],
        'Red': ['Red', 'FireBrick1', 'Red3', 'Crimson', 'Light Coral'],
        'Teal': ['Teal', 'Dark Cyan', 'Light Sea Green', 'Medium Turquoise', 'Medium Spring Green'],
        'White': ['White', 'Floral White', 'Snow', 'White Smoke', 'Ghost White'],
        'Yellow': ['Yellow', 'Yellow2', 'Gold', 'Gold2', 'Dark Goldenrod1']}

    #Get the coresponding swatches
    swatches = color_swatches.get(user_color, [])

    #Colors that need a black highlight for readability
    highlight_colors = ['White', 'Floral White', 'Snow', 'White Smoke', 'Ghost White', 'Yellow', 'Yellow2']

    #Function for selecting swatch
    def select_swatch(selected_color):
        if selected_color in highlight_colors:
            fg_color = selected_color
            bg_color = 'black'
        else:
            fg_color = selected_color
            bg_color = 'white'

        #Update the popup window
        for widget in popup.winfo_children():
            widget.destroy()

        selected_label = tk.Label(popup, text = f"You selected {selected_color}", font = ('calibre', 14, 'bold'), fg = fg_color, bg = bg_color)
        selected_label.pack(pady = 10)

        #Add confirmation message
        confirmation_message = tk.Label(popup, text = "Do you want to confirm your selection?", font = ('calibre', 12, 'normal'))
        confirmation_message.pack(pady = 10)

        #Add Back button to go back
        back_button = tk.Button(popup, text = "Back", font = ('calibre', 12, 'normal'), command = lambda: display_swatches(popup, user_color))
        back_button.pack(side = tk.LEFT, padx = 20, pady = 10)

        #Add Confirm button and save to database
        confirm_button = tk.Button(popup, text = "Confirm", font = ('calibre', 12, 'normal'), command = lambda: save_to_database(selected_color))
        confirm_button.pack(side = tk.RIGHT, padx = 20, pady = 10)

    #Create buttons for each swatch
    for color in swatches:
        swatch_button = tk.Button(popup, bg = color, width = 18, height = 3, command = lambda c = color: select_swatch(c))
        swatch_button.pack(pady = 5)

#Function that handles color selection
def getColor():
    user_color = color_pick.get()
    popup = tk.Toplevel(main)
    popup.title("Your Selection")

    #Display swatches
    display_swatches(popup, user_color)

#Add Submit button
submit_button = tk.Button(main, text = "Submit", font = ('calibre', 14, 'normal'), command = getColor)

#Layout
label_color.pack()
color_pick.pack()
submit_button.pack()

color_pick.current(0)
main.mainloop()

db_path = 'color_selections.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT * FROM selections")
rows = cursor.fetchall()

print("Database:")
for row in rows:
    print(row)

conn.close()

Saved Gold2 to database.
Database:
(1, 'Aquamarine')
(2, 'Tomato')
(3, 'Forest Green')
(4, 'Teal')
(5, 'Light Slate Gray')
(6, 'Gold')
(7, 'FireBrick1')
(8, 'Peru')
(9, 'Hot Pink')
(10, 'Deep Sky Blue')
(11, 'Gold2')
